In [1]:
from iteru import *

In [ ]:
# Map = Map()
# Map

In [ ]:
# startYear = 2020
# startMonth =1
# startDay = 1
# endYear = 2022
# endMonth = 3
# endDay = 17

In [ ]:
# SAR_col= S1_SAR_col(GERD_aoi,startYear,startMonth,startDay,endYear,endMonth,endDay,temp_freq=None)

In [ ]:
# SAR=SAR_col[0];dates_sequences = SAR_col[1]

In [ ]:
# url = SAR_timeseries_url(SAR,GERD_aoi,vis_method='water_mask_only')

In [ ]:
# out_gif = get_gif(url)
# add_text_to_gif(out_gif,dates_sequences, dates_font_size = 25, dates_font_color = 'white', framesPerSecond = 3)
# display_gif(out_gif)

In [ ]:
# water_vectors = SAR.map(filterSpeckles).map(water_classify).map(water_to_vector)

In [ ]:
#water_area = [area /(1e6) for area in water_vectors.aggregate_array('Area').getInfo()]

In [ ]:
# dems = water_vectors.map(max_water_ele)

In [ ]:
# max_elev = [elev for elev in dems.aggregate_array('Maximum_water_elevation').getInfo()]

In [ ]:
# volume_stats = dems.map(water_vol)

In [ ]:
# ele_sum = volume_stats.aggregate_array('Elevation_sum').getInfo()

In [ ]:
# pixel_num  = volume_stats.aggregate_array('Pixels_number').getInfo()

In [ ]:
# volume = [((water_level*pixles_count-elevations_sum)*900)/(1e9) for water_level,elevations_sum , pixles_count in zip(max_elev,ele_sum,pixel_num)]

In [2]:
GERD_SAR_timelaspe(  aoi = GERD_aoi,
                     startYear = 2020,
                     startMonth = 6,
                     startDay = 1,
                     endYear = 2022,
                     endMonth = 3,
                     endDay = 10,
                     temp_freq=None, 
                     vis_method = 'rgb',
                     crs='EPSG:3857',
                     dimensions=900,
                     dates_font_size = 20,
                     copywrite_font_size = 12,
                     dates_font_color = 'black',
                     copywrite_font_color = 'black',
                     framesPerSecond = 3,                     
                    )

OSError: cannot open resource

In [ ]:
# stats_dict = GERD_water_stats(  aoi=GERD_aoi,
#                      startYear=2020,
#                      startMonth=6,
#                      startDay=1,
#                      endYear=2022,
#                      endMonth=3,
#                      endDay=10,
#                      temp_freq='monthly',
#                      water_area=True,
#                      water_level=True,
#                      water_volume=True )

In [ ]:
# stats_dict['Image_date']